In [105]:
import STARTHER
import nvdbapiv3
import pandas as pd
import geopandas as gpd
from shapely import wkt
import xmltodict
from xml.parsers.expat import ExpatError
import json

# Analyse av systemobjekt

In [65]:
sok = nvdbapiv3.nvdbFagdata( 794)
mydf = pd.DataFrame( sok.to_records( relasjoner=True ))

Eksport av 134267 objekter kommer til å ta tid...
Objekt 1000 av 134267
Objekt 5000 av 134267
Objekt 10000 av 134267
Objekt 20000 av 134267
Objekt 30000 av 134267
Objekt 40000 av 134267
Objekt 50000 av 134267
Objekt 60000 av 134267
Objekt 70000 av 134267
Objekt 80000 av 134267
Objekt 90000 av 134267
Objekt 100000 av 134267
Objekt 110000 av 134267
Objekt 120000 av 134267
Objekt 130000 av 134267
Manglende geometri-element for 7 vegobjekter fra dette søket
nvdbFagdata: Søkeobjekt for vegobjekter fra NVDB api V3
ObjektType: 794 Systemobjekt
Filtere
{}
Parametre som styrer responsen:
{
    "inkluder": [
        "alle"
    ]
}
Statistikk fra NVDB api V3
{
    "antall": 134267,
    "lengde": 0.0
}
Pagineringsinfo: Antall objekt i databuffer= 0
{
    "antall": 1000,
    "hvilken": 1,
    "meredata": false,
    "initielt": false,
    "dummy": false
}
fra miljø https://nvdbapiles-v3.atlas.vegvesen.no/


In [69]:
def hvilkeForeldre( rel:dict): 
    returnval = '-99 Uten foreldre'
    
    if not pd.isnull( rel ) and 'foreldre' in rel: 
        temp = [ str( x['type']['id'] ) + ' ' + str( x['type']['navn'] )  for x in rel['foreldre'] ]
        temp.sort()
        returnval = ','.join( temp )
        
    return returnval 

In [70]:
mydf['hvilkeForeldre'] = mydf['relasjoner'].apply( hvilkeForeldre )

In [71]:
mydf['hvilkeForeldre'].value_counts( dropna=False)

95 Skiltpunkt                     44338
79 Stikkrenne/Kulvert             39625
83 Kum                            24949
87 Belysningspunkt                13316
98 Referansestolpe                 4921
7 Gjerde                           1349
20 Kantstolper/Refleks              870
-99 Uten foreldre                   801
174 Gangfelt                        742
25 Leskur                           632
3 Skjerm                            580
27 Renovasjon                       574
103 Fartsdemper                     419
23 Vegbom                           166
342 Trafikkspeil                    129
28 Utemøbler                        116
90 Signalpunkt                      111
519 Vegoppmerking, tverrgående      107
181 Lysmast                          99
502 Skap, teknisk                    82
465 Utgår_Matepunkt                  81
623 Stativ for turistinfo            63
22 Ferist                            62
163 Kamera                           23
24 Skiltportal                       22


In [90]:
normaliser_beskrivelse( mydf.iloc[4063]['Beskrivelse'] )

[{'vegreg_frapos.gnss_data.talker': 'GN',
  'vegreg_frapos.gnss_data.mode': 'DD',
  'vegreg_frapos.gnss_data.antsat': '17',
  'vegreg_frapos.gnss_data.cq': '0.301',
  'vegreg_frapos.gnss_data.nord': '6567637.586',
  'vegreg_frapos.gnss_data.oest': '-33195.613',
  'vegreg_frapos.gnss_data.hoeyde': '14.465',
  'vegreg_frapos.gnss_data.avstand': '0.998'}]

In [74]:
xmltodict.parse( mydf.iloc[4063]['Beskrivelse'] )

{'vegreg_frapos': {'gnss_data': {'talker': 'GN',
   'mode': 'DD',
   'antsat': '17',
   'cq': '0.301',
   'nord': '6567637.586',
   'oest': '-33195.613',
   'hoeyde': '14.465',
   'avstand': '0.998'}}}

In [68]:
pd.json_normalize(  xmltodict.parse('<vegreg_frapos/>'), sep='_').to_dict(orient='records')

[{'vegreg_frapos': None}]

In [75]:
mydf['Beskrivelse'].fillna( '<vegreg_frapos/>', inplace=True)

In [114]:
def dekodbeskrivelse( besk:str ): 
    
    try: 
        mydict = xmltodict.parse(besk )
    except ExpatError as e: 
        # print(f"Feilsituasjon {e} for input {besk} " )
        return { 'ExpatError' : str(e) }

    else: 
        norm = pd.json_normalize( mydict ).to_dict( orient='records')

        if isinstance( norm, list) and len( norm) == 1 and isinstance( norm[0], dict): 
            return norm[0]
        else:
            print( f"Klarte ikke dekode {besk} ")
    
    return { 'error' : 'empty'} 

In [113]:
mydf['Beskrivelse_normalisert'] = mydf['Beskrivelse'].apply(  dekodbeskrivelse )

In [116]:
dekodet = pd.DataFrame( list( mydf['Beskrivelse_normalisert']  ))

In [123]:
for mycol in dekodet.columns: 
    print( mycol, '==========\n', dekodet[mycol].value_counts(dropna=False), '\n')

ExpatError ==========
 NaN                                                   121904
unclosed token: line 1, column 36                       7128
unclosed token: line 1, column 251                      2903
unclosed token: line 1, column 250                       638
junk after document element: line 1, column 401          396
junk after document element: line 1, column 402          366
junk after document element: line 1, column 400          228
unclosed token: line 1, column 249                       147
unclosed token: line 1, column 252                       140
junk after document element: line 1, column 399          109
syntax error: line 1, column 0                           105
junk after document element: line 1, column 403           83
junk after document element: line 1, column 398           38
unclosed token: line 1, column 248                        20
junk after document element: line 1, column 404           14
not well-formed (invalid token): line 1, column 46         6
j

# Hvor mange gyldige datarader har vi? 

ExpatError = antall rader som feiler 

Alle andre rader = Antall rader med gyldige data (dvs totalt antall rader - rader med null, nan, none ... )

In [129]:
len( dekodet )  - dekodet.isnull().sum()

ExpatError                                    12335
vegreg_frapos.gnss_data.talker               120654
vegreg_frapos.gnss_data.mode                  93258
vegreg_frapos.gnss_data.antsat               120657
vegreg_frapos.gnss_data.cq                   120657
vegreg_frapos.gnss_data.nord                 120657
vegreg_frapos.gnss_data.oest                 120657
vegreg_frapos.gnss_data.hoeyde               120657
vegreg_frapos.gnss_data.avstand              120657
vegreg_frapos.vegref_forrige.fylke_nr        120445
vegreg_frapos.vegref_forrige.kommune_nr      120445
vegreg_frapos.vegref_forrige.veg_kategori    120445
vegreg_frapos.vegref_forrige.veg_status      120445
vegreg_frapos.vegref_forrige.veg_nr          120445
vegreg_frapos.vegref_forrige.hp              120445
vegreg_frapos.vegref_forrige.meter           120445
vegreg_tilpos.gnss_data.talker                 1168
vegreg_tilpos.gnss_data.mode                   1127
vegreg_tilpos.gnss_data.antsat                 1168
vegreg_tilpo

In [112]:
mydf['lengde_Beskrivelsestekst'] = mydf['Beskrivelse'].apply( lambda x : len( x ) )

In [63]:
 pd.json_normalize(xmltodict.parse(  mydf.iloc[4063]['Beskrivelse'] ), sep='_').to_dict(orient='records') 

[{'vegreg_frapos_gnss_data_talker': 'GN',
  'vegreg_frapos_gnss_data_mode': 'DD',
  'vegreg_frapos_gnss_data_antsat': '17',
  'vegreg_frapos_gnss_data_cq': '0.301',
  'vegreg_frapos_gnss_data_nord': '6567637.586',
  'vegreg_frapos_gnss_data_oest': '-33195.613',
  'vegreg_frapos_gnss_data_hoeyde': '14.465',
  'vegreg_frapos_gnss_data_avstand': '0.998'}]

In [62]:
mydf[ mydf['Beskrivelse'].str.contains('tilpos') ]['hvilkeForeldre'].value_counts()

7 Gjerde                  1117
20 Kantstolper/Refleks     741
3 Skjerm                   473
-99 Uten foreldre           11
Name: hvilkeForeldre, dtype: int64